# Creating the 'feature matrix' for modeling from the trial matrix

This notebook is the working document for the function `create_feature_matrix` where each row represents a trial and contains information about the last `n trials`. The idea is we want to build a model that, using the information about previous decisions, can predict a future decision. 

While the feature matrix was originally designed for modeling, I'm now using it as a more general piece of data to both analyze & model the behavior. It's nice to do both with the same thing, since it allows us to easily compare the models' behavior to the mouse's. 

## Brief & Incomplete history

**v0** summary data for last 10 trials, individual trial data for previous 5, streak, port decision. Includes trial duration for the 0th trial (ie the trial we are trying to predict).

**v1** individual trial data for last 10 excluding the 0th trial duration. includes both 0th port decision and 0th switch boolean variable.

**v2** added block trial number

**v3** added p(high) port boolean

**v4** added reward for current trial

# Random Thoughts

## block trial

This is something the animal doesn't know, which makes it inherently different than all the other features. We want a model that predicts the mouse's behavior strictly using information the mouse also has access to, and in that way, this feature violates that. 

However, its a useful metric for analyzing behavior. And, if it *does* prove to be useful in modeling the data, that in of itself is an interesting result. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp

In [2]:
bp.

'/Users/shayneufeld/GitHub/mouse_bandit/jupyter_notebooks'

In [2]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
import bandit_preprocessing as bp

In [3]:
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

In [4]:
trials = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/data/80_20_trial_data/dumble_harry/dumble_070716_8020_trials.csv',names=columns)

In [5]:
trials.head(10)

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given
0,36.037,1.801,0.439,2,0.2,0.8,1
1,38.218,1.828,0.353,2,0.2,0.8,1
2,40.551,1.917,0.416,2,0.2,0.8,1
3,43.372,2.160,0.661,2,0.2,0.8,1
4,46.131,2.411,0.348,2,0.2,0.8,1
5,49.824,3.286,0.407,2,0.2,0.8,0
6,52.692,2.405,0.463,2,0.2,0.8,1
7,55.438,2.379,0.367,2,0.2,0.8,1
8,57.997,2.234,0.325,2,0.2,0.8,1
9,60.612,2.133,0.482,2,0.2,0.8,0


In [14]:
mouse_id = 'dumble'
session_id = '070716'

# Important Note:

For now, will only consider 10th trial and onwards for simplicity. 
**May want to revisit this for model that builds its own features for simulating behavior session**

In [138]:
n_trials = trials.shape[0]
n_trials

n_summary = 10;
n_indi = 5;

In [1]:
df = trials.copy()
n_trials = df.shape[0]

n_indi = 10;

num_cols = 2 + 1+ 4*n_indi+ 1 + 4
#2 streak cols, 1 block trial col, 4 cols for each past trial, 1 for current trial + 4 decision/switch/higher p port/reward!
feature_matrix = np.zeros((n_trials-n_indi,num_cols))

NameError: name 'trials' is not defined

In [140]:
feature_matrix.shape

(547, 47)

In [147]:
block_starts = np.zeros(trials.shape[0])

block_starts[1:] = np.diff(trials['Right Reward Prob'].values) != 0

In [148]:
block_starts.shape

(557,)

In [112]:
feature_matrix.shape

(547, 47)

In [149]:
for j,i in enumerate(np.arange(n_indi,n_trials)):
    
    #extract the 'n_summary' trials we need to consider. Assume that n_summary > n_indi
    past_trials = df.iloc[i-n_indi:i]
    
    '''
    Mouse ID
    '''
    # will be added after (since its a string)
    
    '''
    Session ID
    '''
    # will be added after (since its a string)
    
    '''
    Block Number
    '''
    # will be added after all at once (since can be calculated for every trial with single line)
    
    '''
    Block Trial Number
    '''
    if (j == 0): #first block number will be the sum
        if (np.sum(block_starts[:n_indi]) == 0):
        #then we are still in the first block, and we simply are n_indi trials in
            feature_matrix[j,0] = n_indi+1
        else:
            feature_matrix[j,0] = n_indi - np.where(block_starts[:n_indi]==True) + 1
            # i.e. 10 - 5 + 1 = 6 or 10 - 8 + 1 = 3. it works. 
    elif block_starts[i]: #if block_starts[j] == True, start counting from 0
        feature_matrix[j,0] = 0 
    else:
        feature_matrix[j,0] = feature_matrix[j-1,0] + 1
        
    '''
    PORT STREAK
    
    approach: take the derivative of the 'port poked' variabe. Streak is number of [0s + 1] (from the end). 
    the valence of the streak is the sign of first non-zero entry (from the end)
    '''
    streakP_vec = np.flipud(np.diff(past_trials['Port Poked'].values)) #reverse order of array so end is
    #at the front. This makes it easier to find the first non-zero entry
    streakP_len = np.nonzero(streakP_vec)[0]
    
    if len(streakP_len) == 0: #have to deal with case where streak is all 10 previous trials!
        feature_matrix[j,1] = 10
    #otherwise, streak is less then 10 trials and things are simpler.
    else:
        streakP_len = streakP_len[0]
        feature_matrix[j,1] = (streakP_len+1)
    
    
    '''
    REWARD STREAK
    
    approach: take the derivative of the reward boolean. Streak is number of [0s + 1] (from the end). 
    the valence of the streak is the sign of first non-zero entry (from the end)
    '''
    streakR_vec = np.flipud(np.diff(past_trials['Reward Given'].values)) #reverse order of array so end is
    #at the front. This makes it easier to find the first non-zero entry
    streakR_len = np.nonzero(streakR_vec)[0]
    
    if len(streakR_len) == 0: #have to deal with case where streak is all 10 previous trials!
        streakR_len = 10
        if np.sum(past_trials['Reward Given'].values > 0):
            streakR_sign = 1
        else:
            streakR_sign = -1
    #otherwise, streak is less then 10 trials and things are simpler.
    else:
        streakR_len = streakR_len[0]
        streakR_sign = streakR_vec[streakR_len]
    
        feature_matrix[j,2] = (streakR_len+1)*streakR_sign
        
    '''
    INDIVIDUAL TRIALS
    '''
    k = 3
    for icol,itrial in enumerate(np.arange(n_indi,0,-1)):
        
        past_trial = past_trials.iloc[-itrial,:]
        
        #which port
        if past_trial['Port Poked'] == 1:
            feature_matrix[j,k] = 0
        elif past_trial['Port Poked'] == 2:
            feature_matrix[j,k] = 1
        else:
            print('Error port not Left or Right')
        k += 1
        
        #reward given:
        feature_matrix[j,k] = past_trial['Reward Given']
        k += 1
        
        #ITI
        feature_matrix[j,k] = past_trial['Since last trial (s)']
        k += 1
        
        #trial time
        feature_matrix[j,k] = past_trial['Trial Duration (s)']
        k += 1
    
    '''
    CURRENT TRIAL
    '''
    current_trial = df.iloc[i,:]
    feature_matrix[j,k] = current_trial['Since last trial (s)']
    k += 1

    '''
    DECISION
    '''
    if current_trial['Port Poked'] == 1:
        feature_matrix[j,k] = 0
    elif current_trial['Port Poked'] == 2:
        feature_matrix[j,k] = 1
    else:
        print('Error decision port not Left or Right')
    
    k += 1
    '''
    SWITCH
    '''
    feature_matrix[j,k] = np.abs((current_trial['Port Poked'] - df.iloc[i-1]['Port Poked']))
    
    k+= 1
    
    '''
    p(high) port
    '''
    
    if ((current_trial['Right Reward Prob'] > current_trial['Left Reward Prob']) & (current_trial['Port Poked'] == 1)):
        feature_matrix[j,k] = 1
    elif ((current_trial['Right Reward Prob'] < current_trial['Left Reward Prob']) & (current_trial['Port Poked'] == 2)):
        feature_matrix[j,k] = 1
    else:
        feature_matrix[j,k] = 0
    
    k+=1
    
    '''
    REWARD
    '''
    feature_matrix[j,k] = current_trial['Reward given']

[ 22.      8.      6.      0.      0.      2.169   0.395   0.      0.
   3.838   0.469   1.      1.      1.846   0.378   1.      0.      1.527
   0.416   1.      1.      1.635   0.387   1.      1.      1.32    0.341
   1.      1.      2.234   0.621   1.      1.      2.604   0.326   1.      1.
   3.345   0.406   1.      1.      2.508   0.531   3.746   1.      0.      1.   ]


In [142]:
i

556

In [143]:
block_starts.shape

(556,)

In [109]:
current_trial = df.iloc[50:100,:]
current_trial

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given
50,211.11,3.268,0.311,2,0.2,0.8,1
51,213.91,2.485,0.310,2,0.2,0.8,1
52,216.94,2.697,0.332,2,0.2,0.8,1
53,219.58,2.223,0.418,2,0.2,0.8,0
54,221.24,1.381,0.286,2,0.2,0.8,1
55,223.89,2.241,0.403,2,0.2,0.8,1
56,226.89,2.672,0.332,2,0.2,0.8,1
57,229.80,2.573,0.335,2,0.2,0.8,1
58,232.37,2.280,0.291,2,0.2,0.8,0
59,234.08,1.428,0.280,2,0.2,0.8,0


In [150]:
'''
convert into feature dataframe
'''
feature_names = [
                'Block Trial',
                'Port Streak',
                'Reward Streak',
                '10_Port','10_Reward','10_ITI','10_trialDuration',
                '9_Port','9_Reward','9_ITI','9_trialDuration',
                '8_Port','8_Reward','8_ITI','8_trialDuration',
                '7_Port','7_Reward','7_ITI','7_trialDuration',
                '6_Port','6_Reward','6_ITI','6_trialDuration',
                '5_Port','5_Reward','5_ITI','5_trialDuration',
                '4_Port','4_Reward','4_ITI','4_trialDuration',
                '3_Port','3_Reward','3_ITI','3_trialDuration',
                '2_Port','2_Reward','2_ITI','2_trialDuration',
                '1_Port','1_Reward','1_ITI','1_trialDuration',
                '0_ITI',
                'Decision',
                'Switch',
                'Higher prob port'
                 ]

## Add mouse ID and session ID

In [155]:
d = {'Mouse ID':mouse_id,'Session ID':session_id}
feature_df = pd.DataFrame(data=d,index=range(feature_matrix.shape[0]))

In [156]:
feature_df = pd.concat([feature_df,pd.DataFrame(data=feature_matrix,index=None,columns=feature_names)],axis=1)
feature_df.head(80)

,Mouse ID,Session ID,Block Trial,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,10_trialDuration,9_Port,...,2_ITI,2_trialDuration,1_Port,1_Reward,1_ITI,1_trialDuration,0_ITI,Decision,Switch,Higher prob port
0,dumble,070716,11.0,10.0,-1.0,1.0,1.0,1.801,0.439,1.0,...,2.234,0.325,1.0,0.0,2.133,0.482,1.411,1.0,0.0,1.0
1,dumble,070716,12.0,10.0,-2.0,1.0,1.0,1.828,0.353,1.0,...,2.133,0.482,1.0,0.0,1.411,0.435,2.169,0.0,1.0,0.0
2,dumble,070716,13.0,1.0,-3.0,1.0,1.0,1.917,0.416,1.0,...,1.411,0.435,0.0,0.0,2.169,0.395,3.838,0.0,0.0,0.0
3,dumble,070716,14.0,2.0,-4.0,1.0,1.0,2.160,0.661,1.0,...,2.169,0.395,0.0,0.0,3.838,0.469,1.846,1.0,1.0,1.0
4,dumble,070716,15.0,1.0,1.0,1.0,1.0,2.411,0.348,1.0,...,3.838,0.469,1.0,1.0,1.846,0.378,1.527,1.0,0.0,1.0
5,dumble,070716,16.0,2.0,-1.0,1.0,0.0,3.286,0.407,1.0,...,1.846,0.378,1.0,0.0,1.527,0.416,1.635,1.0,0.0,1.0
6,dumble,070716,17.0,3.0,1.0,1.0,1.0,2.405,0.463,1.0,...,1.527,0.416,1.0,1.0,1.635,0.387,1.320,1.0,0.0,1.0
7,dumble,070716,18.0,4.0,2.0,1.0,1.0,2.379,0.367,1.0,...,1.635,0.387,1.0,1.0,1.320,0.341,2.234,1.0,0.0,1.0
8,dumble,070716,19.0,5.0,3.0,1.0,1.0,2.234,0.325,1.0,...,1.320,0.341,1.0,1.0,2.234,0.621,2.604,1.0,0.0,1.0
9,dumble,070716,20.0,6.0,4.0,1.0,0.0,2.133,0.482,1.0,...,2.234,0.621,1.0,1.0,2.604,0.326,3.345,1.0,0.0,1.0
